In [4]:
import torch
import numpy as np
import torch.nn as nn
from tqdm.notebook import tqdm
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import get_cosine_schedule_with_warmup

In [17]:
# 모델 불러오기 (CPU)
model_parameters = torch.load(
    "./models/bidirectional_classifier.pth",
    map_location=torch.device('cpu')
)

In [37]:
test_input = torch.load('./processed_data/test_input.pt')
vocab_number = 80096
label_number = 7
batch_size = 32
n_hidden = 300

In [33]:
class TextDataset(Dataset):
    def __init__(self,tokens, labels=None ,is_test=False):
        super().__init__()
        self.tokens = tokens
        self.labels = labels
        self.is_test = is_test
    
    def __len__(self):
        return len(self.tokens)    
    
    def __getitem__(self, idx):
        input = self.tokens[idx]
        label = None
        if not self.is_test:
            label = self.labels[idx]
        return (input, label) if not self.is_test else (input)

In [34]:
dataset_test = TextDataset(tokens=test_input, is_test=True)
loader_test = DataLoader(dataset=dataset_test, batch_size=batch_size, shuffle=False,drop_last=True) 

In [35]:
class TextLSTM(nn.Module):
    def __init__(self):
        super(TextLSTM, self).__init__()
                                      # 80096 + <unk,padding> = 80097
        self.embedding = nn.Embedding(vocab_number, n_hidden)
        self.lstm = nn.LSTM(
        input_size=n_hidden, 
        hidden_size=n_hidden, 
        num_layers=1,
        dropout=0.5, # drop out -> 0.5
        bidirectional=True,
        )
        self.fc = nn.Linear(2*n_hidden, label_number)
        
    def forward(self, hidden_and_cell,X):
        X = self.embedding(X)
        X = X.transpose(0,1)
        outputs, (hidden_forward, hidden_backward) = self.lstm(X, hidden_and_cell)
        
        # forward, backward 방향의 hidden state 를 합침
        outputs = torch.cat(
        (hidden_forward[-1], hidden_backward[-1]), dim=1
        )  
        outputs = self.fc(outputs)
        
        return outputs

In [36]:
model = TextLSTM()
model.load_state_dict(model_parameters)

c:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


RuntimeError: Error(s) in loading state_dict for TextLSTM:
	size mismatch for embedding.weight: copying a param with shape torch.Size([80096, 300]) from checkpoint, the shape in current model is torch.Size([80097, 300]).

In [19]:
hidden = torch.zeros(2, batch_size, n_hidden, requires_grad=True)
cell = torch.zeros(2, batch_size, n_hidden, requires_grad=True)

final_preds = []

for tokens in loader_test: # batch 갯수때문에 복수형
    tokens = tokens
    outputs = model.forward((hidden,cell),tokens)
    pred = torch.max(outputs.cpu(), dim=1)[1].numpy() # 예측값

    final_preds.extend(pred)

In [22]:
print(len(final_preds))

13472


In [29]:
for a in loader_test:
    print(a.shape)
    break

torch.Size([32, 121])


In [30]:
print(hidden.shape)
print(cell.shape)

torch.Size([2, 32, 300])
torch.Size([2, 32, 300])


In [40]:
last_input = test_input[-32:]

for row,tmp_input in enumerate(last_input):
    for column,value in enumerate(tmp_input):
        if value>=80096:
            last_input[row][column] = 0

In [41]:
# 마지막 32개에 맞춰서 마지막 테스트 데이터 예측
import numpy as np
print(last_input.shape)
outputs = model.forward((hidden,cell),last_input)
last_pred = torch.max(outputs.cpu(), dim=1)[1].numpy() # 예측값
print(last_pred)

torch.Size([32, 121])
[3 3 0 6 1 3 2 1 1 6 4 1 6 0 1 3 1 5 0 1 2 3 1 3 3 6 5 1 0 4 1 6]


In [42]:
part_last_pred = last_pred[-19:]
final_preds.extend(part_last_pred)

In [43]:
import pandas as pd

submission_df = pd.read_csv("input/dataset/test.csv")
submission_df['label']= final_preds
submission_df = submission_df[['ID','label']]

In [44]:
submission_df.to_csv('./output/jhkim_2.csv',index=False)